# Install

In [ ]:
!pip install --upgrade pip
!pip install pandas
!pip install awswrangler
!pip install pyspellchecker
!pip install emoji
!pip install swifter
!pip install nltk
!pip install wordcloud
!conda uninstall -y TBB
!pip install tbb>=2019.0
!pip install bertopic
!pip install gensim
!pip install unidecode
!pip install --upgrade sentence-transformers
!pip install ipywidgets

# Import

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from sklearn.compose import ColumnTransformer

import re
import nltk
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker
import emoji
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

import unidecode
from dataclasses import dataclass
import awswrangler as wr
from wordcloud import WordCloud

from bertopic import BERTopic
import pickle
import joblib

import torch
from torch.nn.functional import cosine_similarity
import transformers

import boto3


from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans

from sklearn.feature_extraction.text import CountVectorizer

from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer

from itertools import product

from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration

import os


nltk.download('stopwords')
nltk.download('punkt')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

## Pipeline

In [ ]:
@dataclass
class emojiTransformer(BaseEstimator, TransformerMixin):
    column: str
    
    def deEmojify(self, text) : 
        regrex_pattern = re.compile(pattern = "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags = re.UNICODE)
        return regrex_pattern.sub(r'',text)
    
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        emoji_package_removal = X[self.column].apply(lambda x: emoji.replace_emoji(x, replace=''))
        # regex_removal = pd.DataFrame(X.apply(lambda x: self.deEmojify(text=x)))
        return pd.DataFrame(emoji_package_removal, columns=[self.column])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)
    
@dataclass
class lowercaseTransformer(BaseEstimator, TransformerMixin):  
    column: str
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return pd.DataFrame(X[self.column].apply(lambda x: x.lower()), columns=[self.column])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)
    
@dataclass
class accentsTransformer(BaseEstimator, TransformerMixin):  
    column: str
    
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return pd.DataFrame(X[self.column].apply(lambda x: unidecode.unidecode(x)),
                            columns=[self.column])
    def fit_transform(self, X, y=None):
        return self.transform(X)
    
@dataclass
class numberFilterTransformer(BaseEstimator, TransformerMixin):  
    column: str
    
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return pd.DataFrame(X[self.column].apply(lambda x: re.sub(r'\d+', '', x)),
                            columns=[self.column])
    def fit_transform(self, X, y=None):
        return self.transform(X)
    
@dataclass
class punctuationFilterTransformer(BaseEstimator, TransformerMixin):  
    column: str
    
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return pd.DataFrame(X[self.column].apply(lambda x: re.sub(r'[^\w\s]','', x)),
                            columns=[self.column])
    def fit_transform(self, X, y=None):
        return self.transform(X)
    
@dataclass
class spellTransformer(BaseEstimator, TransformerMixin):  
    column: str
    
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        spell = SpellChecker(language='pt') 
        return pd.DataFrame(X[self.column].swifter.apply(lambda x: spell.correction(x)))
    def fit_transform(self, X, y=None):
        return self.transform(X)
    
@dataclass
class stopwordsTransformer(BaseEstimator, TransformerMixin):  
    column: str
    
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        stopwords = nltk.corpus.stopwords.words('portuguese')
        return pd.DataFrame(X[self.column].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords])),
                            columns=[self.column])
    def fit_transform(self, X, y=None):
        return self.transform(X)
    
@dataclass
class wordTokenizerTransformer(BaseEstimator, TransformerMixin):  
    column: str
    ngram: int=1
    
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return pd.DataFrame(X[self.column].apply(lambda x: word_tokenize(x)),
                            columns=[self.column])
    def fit_transform(self, X, y=None):
        return self.transform(X)
    
@dataclass
class removerTransformer(BaseEstimator, TransformerMixin):  
    column: str
    words = ['ok',
             'okay',
             'hum']
    
    def fit(self, X, y=None):
        return self
    def transform(self, X):

        return pd.DataFrame(X[self.column].apply(lambda x: ' '.join([word for word in x.split() if word not in self.words])),
                            columns=[self.column])
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [ ]:
transformation_column = 'ds_message'
bert_steps = [
 ('remove_emojis', emojiTransformer(transformation_column)),
 ('lower_case', lowercaseTransformer(transformation_column)),
 ('accents', accentsTransformer(transformation_column)),
 ('number_filter', numberFilterTransformer(transformation_column)),
 ('punctuation_filter', punctuationFilterTransformer(transformation_column)),
 # ('remove_words', removerTransformer(transformation_column)),
 ('stopwords', stopwordsTransformer(transformation_column)),
]

bert_pipe = Pipeline(steps=bert_steps)

# Global config

In [ ]:
min_date = '2023-01-01'
max_date = '2024-02-01'

## Query empréstimos

In [ ]:
query = f"""

  
"""
# Define staging_path
staging_path = 

# Define boto3 session
boto3_session = boto3.Session(region_name="sa-east-1")

# Clean temporary folder
wr.s3.delete_objects(staging_path, boto3_session=boto3_session)

# Run query
raw_data = wr.athena.read_sql_query(
    sql=query,
    database=None,
    ctas_approach=False,
    s3_output=staging_path,
)
wr.s3.delete_objects(staging_path, boto3_session=boto3_session)


In [ ]:
# Define staging_path
staging_path = ""

# Define boto3 session
boto3_session = boto3.Session(region_name="sa-east-1")

# Clean temporary folder
wr.s3.delete_objects(staging_path, boto3_session=boto3_session)

query = """"""

# Run query
tmp = wr.athena.read_sql_query(
    sql=query,
    database=None,
    ctas_approach=False,
    s3_output=staging_path,
)
wr.s3.delete_objects(staging_path, boto3_session=boto3_session)


## Methods

In [ ]:
def expand_dict_values(dictionary):
    keys, values = zip(*dictionary.items())
    expanded_values = list(product(*values))

    expanded_dicts = []
    for val in expanded_values:
        expanded_dicts.append(dict(zip(keys, val)))

    return expanded_dicts

def merge_consecutive_chats(raw_data, tag):
    df = raw_data[raw_data.ds_tags==tag].copy()
    df = df.sort_values(['id_issue','id_customer','dt_envio_mensagem','dt_criacao_chat'])
    display(df.shape)
    df['shift'] = (df['ds_entidade'] != df['ds_entidade'].shift())
    df['group'] = df['shift'].apply(lambda x: 0 if pd.isna(x) or x!=True else 1).cumsum()
    display(df.shape)
    df = df.groupby(['id_issue','id_customer', 'ds_entidade', 'group','ds_tags']).agg(
        ds_message=('ds_message', ' '.join),
        dt_criacao_chat=('dt_criacao_chat', 'min'),
        dt_envio_mensagem=('dt_envio_mensagem', 'min')
    )
    display(df.shape)
    df = df.sort_values(['id_issue','id_customer','group','dt_criacao_chat','dt_envio_mensagem']).reset_index()
    display(df.shape)
    df = df[df['ds_entidade']!='automação']
    return df

def write_model(topic_model,tag,params,entity,outlier=False):
    p = params
    if outlier:
        with open(f"{tag}_ngram_{p['n_gram_range'][0]}{p['n_gram_range'][1]}_nr_{p['nr_topics']}_min_{p['min_topic_size']}_{entity}_outlier.pkl", 'wb') as file:    
                pickle.dump(topic_model, file)
    else:
        with open(f"{tag}_ngram_{p['n_gram_range'][0]}{p['n_gram_range'][1]}_nr_{p['nr_topics']}_min_{p['min_topic_size']}_{entity}.pkl", 'wb') as file:    
                pickle.dump(topic_model, file)
                


def load_models(tag,
                entity,
                dimension_params,
                clustering_params,
                vectorizer_params,
                ctfidf_params,
                best_param = True
               ):
    count = 0
    for _dim_model, _dim_model_params in dimension_params.items():
        for _d_p in expand_dict_values(_dim_model_params):
            for _cluster_model, _cluster_model_params in clustering_params.items():
                for _c_p in expand_dict_values(_cluster_model_params):
                    for _vec_model, _vec_params in vectorizer_params.items():
                        for _v_p in expand_dict_values(_vec_params):                    
                            for _idf_model, _idf_params in ctfidf_params.items():
                                for _i_p in expand_dict_values(_idf_params):
                                    try:
                                        print('Loading...')
                                        file_path = f"models/{unidecode.unidecode(tag.lower().replace(' ','_').replace('-','').replace('|',''))}__"+\
                                              f"{entity}__"+\
                                              f"m__d_{_dim_model}_{'_'.join([str(i) for i in list(_d_p.values())])}"+\
                                              f"__c_{_cluster_model}_{'_'.join([str(i) for i in list(_c_p.values())])}"+\
                                              f"__v_{_vec_model}_{'_'.join([str(i) for i in list(_v_p.values())])}"+\
                                              f"__i_{_idf_model}_{'_'.join([str(i) for i in list(_i_p.values())])}.pkl"
                                        print(file_path)
                                        with open(file_path, 'rb') as file:
                                            topic_model = pickle.load(file)
                                            display('Total chats: ', topic_model.get_topic_info()['Count'].sum())
                                            topic_info = topic_model.get_topic_info()
                                            # mask = ~topic_info.Representation.apply(lambda x: any(item in x for item in post_filter))
                                            # display(topic_info[mask].head(7))
                                            display(topic_info.head(10))
                                            
                                    except Exception as e: 
                                        print(e)
                                        
                                        
def train_models(X,
                 tag,
                 entity,
                 dimension_params,
                 clustering_params,
                 vectorizer_params,
                 ctfidf_params,
                 count_mode=False
               ):
    count = 0
    for _dim_model, _dim_model_params in dimension_params.items():
        for _d_p in expand_dict_values(_dim_model_params):
            for _cluster_model, _cluster_model_params in clustering_params.items():
                for _c_p in expand_dict_values(_cluster_model_params):
                    for _vec_model, _vec_params in vectorizer_params.items():
                        for _v_p in expand_dict_values(_vec_params):                    
                            for _idf_model, _idf_params in ctfidf_params.items():
                                for _i_p in expand_dict_values(_idf_params):
                                    if count_mode:
                                        count+=1
                                        continue 
                                    print('Creating file... ')
                                    file_path = f"models/{unidecode.unidecode(tag.lower().replace(' ','_').replace('-','').replace('|',''))}__"+\
                                              f"{entity}__"+\
                                              f"m__d_{_dim_model}_{'_'.join([str(i) for i in list(_d_p.values())])}"+\
                                              f"__c_{_cluster_model}_{'_'.join([str(i) for i in list(_c_p.values())])}"+\
                                              f"__v_{_vec_model}_{'_'.join([str(i) for i in list(_v_p.values())])}"+\
                                              f"__i_{_idf_model}_{'_'.join([str(i) for i in list(_i_p.values())])}.pkl"
                                    print(file_path)
                                    
                                    out_file_path = f"models/{unidecode.unidecode(tag.lower().replace(' ','_').replace('-','').replace('|',''))}__"+\
                                              f"{entity}__"+\
                                              f"m__d_{_dim_model}_{'_'.join([str(i) for i in list(_d_p.values())])}"+\
                                              f"__c_{_cluster_model}_{'_'.join([str(i) for i in list(_c_p.values())])}"+\
                                              f"__v_{_vec_model}_{'_'.join([str(i) for i in list(_v_p.values())])}"+\
                                              f"__i_{_idf_model}_{'_'.join([str(i) for i in list(_i_p.values())])}_out.pkl"
                                    
                                    outlier = False
                                    
  
                                    # DIMENSION
                                    if _dim_model == 'umap':
                                        dim_model = UMAP(**_d_p)
                                    if _dim_model == 'svd':
                                        dim_model = TruncatedSVD(**_d_p)
                                    if _dim_model == 'pca':
                                        dim_model = PCA(**_d_p)
                                        
                                    # CLUSTERING
                                    if _cluster_model == 'hdbscan':
                                        cluster_model = HDBSCAN(**_c_p)
                                        outlier = True
                                    if _cluster_model == 'kmeans':
                                        cluster_model = KMeans(**_c_p)
                                    
                                    # VECTORIZER
                                    if _vec_model == 'CountVectorizer':
                                        vec_model = CountVectorizer(**_v_p)
                                    
                                    if _idf_model == 'ctfidf':
                                        idf_model = ClassTfidfTransformer(**_i_p)
                                        
                                    keybert = KeyBERTInspired()
                                    mmr = MaximalMarginalRelevance(diversity=0.3)
                                    
                                    representation_model = {
                                        "KeyBERT": keybert,
                                        "MMR": mmr,
                                    }
                                    
                                    if (os.path.exists(file_path) and os.path.exists(out_file_path)) or (os.path.exists(file_path) and ~outlier):
                                        print('If the data changed please delete the pickle files rm models/*.pkl')
                                        print('File already exists, running next parameter...')
                                        continue
                                    
                                    sentence_model = SentenceTransformer('all-MiniLM-L12-v2')
                                    
                                    topic_model = BERTopic(
                                        embedding_model=sentence_model,
                                        umap_model=dim_model,
                                        hdbscan_model=cluster_model,
                                        vectorizer_model=vec_model,
                                        ctfidf_model=idf_model,
                                        representation_model=representation_model,
                                        top_n_words=10,
                                        nr_topics=50,
                                        verbose=True
                                    )
                                    
                                    if ~os.path.exists(file_path):
                                        print('training model...')
                                        try:
                                            _,__ = topic_model.fit_transform(X['ds_message'])

                                            with open(file_path, 'wb') as file:    
                                                pickle.dump(topic_model, file)
                                            print('created file: ')
                                            print(file_path)
                                        except Exception as e:
                                            print('ERROR: ')
                                            print(e)
                                    
                                    if outlier:
                                        try:
                                            with open(file_path, 'rb') as file:
                                                topic_model = pickle.load(file)

                                            print('outlier')
                                            
                                            pred, prob = topic_model.transform(list(X.ds_message))
                                            topics_df = topic_model.get_topic_info()
                                            pred_df = pd.DataFrame(pred, columns = ['Topic'])
                                            main_df = pd.merge(pred_df, topics_df, on=['Topic'], how='left')
                                            main_df.index = X.index
                                            main_df['Topic'] = main_df['Topic'].astype(str)
                                            merged_df = pd.concat([X,main_df],axis=1)
                                            

                                            pred_out, prob_out = topic_model.fit_transform(list(merged_df[merged_df['Topic']=='-1'].ds_message))
                                            with open(out_file_path, 'wb') as file:    
                                                pickle.dump(topic_model, file)
                                            print('created outlier file: ')
                                            print(out_file_path)

                                        except Exception as e:
                                            print('ERRO OUTLIER')
                                            print(e)
                                        

    if count_mode:
        print('# diferent combinations will be tested:', count)

## Tags

In [ ]:
tags_list = [
    'w3 - credito pessoal | como adquirir',
                    # 'b3 - duvida emprestimo',
                           # 'n3 - emprestimo',
 # 'w3 - credito pessoal | negociacao via app',
                           # 'q3 - empréstimo',
                 # 'w3 - fgts | como adquirir',
        # 'w3 - credito pessoal | amortização',
 #                                    'wb-emp',
 #                                   'wb-fgts',
 #                     'w3 - aviso empréstimo'
]

## Param

In [ ]:
####
#### TRASFOMER
####

# sentence_model = SentenceTransformer('all-MiniLM-L12-v2')
model_path = 'transformers/all-MiniLM-L12-v2'
# sentence_model.save(model_path)
sentence_model = SentenceTransformer(model_path)

####
#### DIMENSIONS
####
umap_model_params = {
    'n_neighbors':[
        15,
        # 30,
                  ],
    'n_components':[
        # 5,
        15,
        # 30,
                   ],
    'min_dist':[0],
    'metric':['cosine'],
}
svd_model_params = {
    'n_components':[
        5,
        10,
        30,
    ],
}
pca_model_params = {
    'n_components':[
        5,
        10,
        30,
    ],
}

dimension_params = {
    'umap':umap_model_params,
    # 'svd':svd_model_params,
    # 'pca':pca_model_params,
}

####
#### CLUSTERS
####
hdbscan_model_params = {
    'min_cluster_size':[
        10,
        # 30,
        35,
        # 50,
    ],
    'metric':[
        'euclidean',
    ],
    'cluster_selection_method':['eom'],
    'prediction_data':[True],
}

kmeans_model_params = {
    'n_clusters':[
        # 10,
        # 30,
        50,
    ]
}
clustering_params = {
    'hdbscan':hdbscan_model_params,
    # 'kmeans':kmeans_model_params,
}

####
#### VECTORIZERS
####
vectorizer_model_params = {
    'ngram_range': [
        (1,1),
        (1,2),
        (1,3),
        # (2,4),
        # (1,4)
    ],
}

vectorizer_params = {
    'CountVectorizer': vectorizer_model_params
}

####
#### CTFIDF
####
ctfidf_model_params = {
    'reduce_frequent_words': [
        # False,
        True,
    ],
}

ctfidf_params = {
    'ctfidf': ctfidf_model_params
}


## Train models

In [ ]:
remove_sentences = {
    'cliente' : [
        'outros assuntos',
        'falar outra coisa',
        'voltar pro comeco',
        'ok',
        'outros assuntos falar outra coisa',
        'opcoes',
        'nao',
        'outro assunto',
        '',
        'falar outro assunto',
        'ja resolvi problema',
        'falar atendente',
        'falar outra coisa emprestimo',
        'certo',
        'oi',
        'nao obrigado',
        'responder mensagem antes',
        'falar outra coisa emprestimo pessoal',
        'opcoes falar outra coisa',
        'ok obrigado',
        'ok obrigada',
        'obrigada',
        'outra coisa sobre fatura',
        'obrigado',
        'nao obrigada',
        'ta',
        'so',
        'ok oi',
        'sim',
        'voltar',
        'outras dicas',
        'outra coisa',
        'ta bom',
        'obg',
        'voltar pro comeco outros assuntos',
        'blz',
        'ola',
        'bom dia',
        'boa tarde',
        'boa noite',
        'outros assuntos voltar',
        'voltar pras opcoes antes',
        'outros assuntos emprestimo fgts',
        'emprestimo pessoal',
        'emprestimo fgts',
        'emprestimo',
        'limite credito',
        'fatura cartao',
        'aumento limite',
        'pix',
        'emprestimo pessoal',
        'tudo bem',
        'entendi',
        'aguardo',
        'esponder mensagem antes emprestimo',
        'nenhuma opcoes',
        'nenhuma dessas',
        'nao ainda duvidas',
        'ainda preciso ajuda',
        'nenhuma opcoes nenhuma dessas',
        'nao ainda duvidas nenhuma opcoes',
        'sim ainda preciso ajuda',
        'ok ok'
        
    ],
    'agente': [
        ''
    ]
}    


remove_pattern = {
    'cliente' : [
        'obrig'
    ],
    'agente': [
        ''
    ]
}
    


In [ ]:
df = merge_consecutive_chats(raw_data, tag)
X = bert_pipe.fit_transform(df[(df['ds_entidade']=='cliente')][['ds_message']]) 
X = X[~X.ds_message.isin(remove_sentences['cliente'])]
X = X[~X.ds_message.str.contains(fr"{'|'.join(remove_pattern['cliente'])}", regex=True)]

In [ ]:
for tag in tags_list:
    print(tag)
    df = merge_consecutive_chats(raw_data, tag)
    display(df.groupby('ds_entidade').count())
    
    if tag[0]=='w' or tag[0]=='n':
        entity = 'cliente'
        print(entity)
        X = bert_pipe.fit_transform(df[(df['ds_entidade']==entity)][['ds_message']]) 
        X = X[~X.ds_message.isin(remove_sentences[entity])]
        X = X[~X.ds_message.str.contains(fr"{'|'.join(remove_pattern[entity])}", regex=True)]
        print('Messages after sentence removal: ',X.shape)

        train_models(X,
                 tag,
                 entity,
                 dimension_params,
                 clustering_params,
                 vectorizer_params,
                 ctfidf_params,
                 count_mode=False)
        
        entity = 'agente'
        print(entity)
        X = bert_pipe.fit_transform(
            df[(df['ds_entidade']==entity)][['ds_message']]) 
        
        train_models(X,
                 tag,
                 entity,
                 dimension_params,
                 clustering_params,
                 vectorizer_params,
                 ctfidf_params,
                 count_mode=False)
    else:
        entity = 'cliente'
        print(entity)
        X = bert_pipe.fit_transform(df[(df['ds_entidade']==entity)][['ds_message']]) 
        # X = X[~X.ds_message.isin(remove_sentences[entity])]
        # X = X[~X.ds_message.str.contains(fr"{'|'.join(remove_pattern[entity])}", regex=True)]
        print('Messages after sentence removal: ',X.shape)

        train_models(X,
                 tag,
                 entity,
                 dimension_params,
                 clustering_params,
                 vectorizer_params,
                 ctfidf_params,
                 count_mode=False)
        
        entity = 'bot'
        print(entity)
        X = bert_pipe.fit_transform(df[(df['ds_entidade']==entity)][['ds_message']]) 
        
        train_models(X,
                 tag,
                 entity,
                 dimension_params,
                 clustering_params,
                 vectorizer_params,
                 ctfidf_params,
                 count_mode=False)


    

## Load models

In [ ]:
for tag in tags_list:
    print(tag)
    if tag[0]=='w' or tag[0]=='n':
        entities = [
            'cliente',
            'agente'
        ]
    else: 
        entities = [
            'cliente',
            'bot'
        ]

    for entity in entities:
        load_models(tag,
                    entity,
                    dimension_params,
                    clustering_params,
                    vectorizer_params,
                    ctfidf_params)
        

## Best params per tag

In [ ]:
best_params_per_tag = {
'w3 - credito pessoal | como adquirir': {
    'cliente' : 'models/w3__credito_pessoal__como_adquirir__cliente__m__d_umap_15_15_0_cosine__c_hdbscan_35_euclidean_eom_True__v_CountVectorizer_(1, 2)__i_ctfidf_True.pkl',
    'agente':'models/w3__credito_pessoal__como_adquirir__agente__m__d_umap_15_15_0_cosine__c_hdbscan_35_euclidean_eom_True__v_CountVectorizer_(1, 2)__i_ctfidf_True.pkl',
},
'b3 - duvida emprestimo': {
    'cliente': 'models/b3__duvida_emprestimo__cliente__m__d_umap_15_15_0_cosine__c_hdbscan_35_euclidean_eom_True__v_CountVectorizer_(1, 2)__i_ctfidf_True.pkl',
    'bot':'models/b3__duvida_emprestimo__bot__m__d_umap_15_15_0_cosine__c_hdbscan_35_euclidean_eom_True__v_CountVectorizer_(1, 2)__i_ctfidf_True.pkl',
},
'n3 - emprestimo':{
    'cliente':'models/n3__emprestimo__cliente__m__d_umap_15_15_0_cosine__c_hdbscan_10_euclidean_eom_True__v_CountVectorizer_(1, 1)__i_ctfidf_True.pkl',
    'agente':'models/n3__emprestimo__agente__m__d_umap_15_15_0_cosine__c_hdbscan_10_euclidean_eom_True__v_CountVectorizer_(1, 1)__i_ctfidf_True.pkl',
},
'w3 - credito pessoal | negociacao via app' :{
    'cliente': 'models/w3__credito_pessoal__negociacao_via_app__cliente__m__d_umap_15_15_0_cosine__c_hdbscan_10_euclidean_eom_True__v_CountVectorizer_(1, 1)__i_ctfidf_True.pkl',
    'agente': 'models/w3__credito_pessoal__negociacao_via_app__agente__m__d_umap_15_15_0_cosine__c_hdbscan_10_euclidean_eom_True__v_CountVectorizer_(1, 3)__i_ctfidf_True.pkl',
},
'q3 - empréstimo' : {
    'cliente':'models/q3__emprestimo__cliente__m__d_umap_15_15_0_cosine__c_hdbscan_10_euclidean_eom_True__v_CountVectorizer_(1, 3)__i_ctfidf_True.pkl',
    'bot':'models/q3__emprestimo__bot__m__d_umap_15_15_0_cosine__c_hdbscan_10_euclidean_eom_True__v_CountVectorizer_(1, 3)__i_ctfidf_True.pkl',
},
'w3 - fgts | como adquirir' : {
    'cliente':'models/w3__fgts__como_adquirir__cliente__m__d_umap_15_15_0_cosine__c_hdbscan_35_euclidean_eom_True__v_CountVectorizer_(1, 2)__i_ctfidf_True.pkl',
    'agente':'models/w3__fgts__como_adquirir__agente__m__d_umap_15_15_0_cosine__c_hdbscan_10_euclidean_eom_True__v_CountVectorizer_(1, 1)__i_ctfidf_True.pkl',
},
'w3 - credito pessoal | amortização': {
    'cliente':'models/w3__credito_pessoal__amortizacao__cliente__m__d_umap_15_15_0_cosine__c_hdbscan_10_euclidean_eom_True__v_CountVectorizer_(1, 3)__i_ctfidf_True.pkl',
    'agente':'models/w3__credito_pessoal__amortizacao__agente__m__d_umap_15_15_0_cosine__c_hdbscan_10_euclidean_eom_True__v_CountVectorizer_(1, 3)__i_ctfidf_True.pkl',
},
# 'wb-emp',
# 'wb-fgts',
}

## Write dfs

In [ ]:
for tag, values in best_params_per_tag.items():
    print(tag)
    error = False
    for ent, file_path in values.items():
        print('entidade:', ent)
        try:
            with open(file_path, 'rb') as file:
                topic_model = pickle.load(file)

            df = merge_consecutive_chats(raw_data, tag)
            if (tag[0]=='w' or tag[0]=='n') and ent=='cliente':
                X = bert_pipe.fit_transform(df[(df['ds_entidade']=='cliente')][['ds_message']])
                X = X[~X.ds_message.isin(remove_sentences[ent])]
                X = X[~X.ds_message.str.contains(fr"{'|'.join(remove_pattern[ent])}", regex=True)]
                print('X dim:',X.shape)
            else:
                X = bert_pipe.fit_transform(df[(df['ds_entidade']==ent)][['ds_message']])
                print('X dim:',X.shape)
            
            pred, prob = topic_model.transform(list(X.ds_message))
            topics_df = topic_model.get_topic_info()
            pred_df = pd.DataFrame(pred, columns = ['Topic'])
            pred_df['prob'] = prob 
            
            main_df = pd.merge(pred_df, topics_df, on=['Topic'], how='left')
            main_df.index = X.index
            main_df['Topic'] = main_df['Topic'].astype(str)
            
        except Exception as e:
            print('ERRO')
            print(e)
            error = True
        
        ## write dataframe
        merged_df = pd.concat([df,main_df],axis=1)
        display(merged_df)
        
        try:
            if 'hdbscan' in file_path:
                    print('outlier')
                    if (tag[0]=='w' or tag[0]=='n') and ent=='cliente':
                        X = bert_pipe.fit_transform(merged_df[merged_df['Topic']=='-1'][['ds_message']])
                        X = X[~X.ds_message.isin(remove_sentences[ent])]
                        X = X[~X.ds_message.str.contains(fr"{'|'.join(remove_pattern[ent])}", regex=True)]
                        print('OUT X dim:',X.shape)
                    else:
                        X = bert_pipe.fit_transform(merged_df[merged_df['Topic']=='-1'][['ds_message']])
                        print('OUT X dim:',X.shape)
                    
                    with open(file_path.split('.')[0]+ '_out' + '.pkl', 'rb') as file:
                        topic_model = pickle.load(file)
                    
                    pred_out, prob_out = topic_model.transform(list(X.ds_message))
                    topics_df = topic_model.get_topic_info()
                    pred_out_df = pd.DataFrame(pred_out, columns = ['Topic'])
                    pred_out_df['prob'] = prob
                    out_df = pd.merge(pred_out_df, topics_df, on=['Topic'], how='left')
                    out_df.index = merged_df[merged_df['Topic']=='-1'].index
                    out_df['Topic'] = '-1_'+ out_df['Topic'].astype(str)
                    out_df['Name'] = '-1_'+ out_df['Name'].astype(str)
                    # display(out_df)
                    
                    # cliente[model_cols].fillna(agente[model_cols]) 
                    cols = ['Topic','Count','Name','Representation','KeyBERT','MMR','Representative_Docs']
                    merged_df.loc[merged_df['Topic']=='-1', cols] = out_df
        except Exception as e:
            print('ERRO OUTLIER')
            print(e)
            error = True
            

            
        if not error:
            merged_df.to_parquet(f'data/{tag}_{ent}_merged.parquet')
       

## Send to dashboard

In [ ]:
query = f"""
"""

# Define staging_path
staging_path = 


# Define boto3 session
boto3_session = boto3_session = boto3.Session(region_name="sa-east-1")

# Clean temporary folder
wr.s3.delete_objects(staging_path, boto3_session=boto3_session)

# Run query
df_ep = wr.athena.read_sql_query(
    sql=query,
    database=None,
    ctas_approach=False,
    s3_output=staging_path,
).drop_duplicates()\
 .add_suffix('_ep')\
 .rename(columns={'cd_perfil_ep_ep':'cd_perfil_ep',
                  'ds_perfil_ep_ep':'ds_perfil_ep'})
wr.s3.delete_objects(staging_path, boto3_session=boto3_session)
                                 

query = f"""

"""
# Define staging_path
staging_path = 

# Define boto3 session
boto3_session = boto3_session = boto3.Session(region_name="sa-east-1")

# Clean temporary folder
wr.s3.delete_objects(staging_path, boto3_session=boto3_session)

# Run query
df_fgts = wr.athena.read_sql_query(
    sql=query,
    database=None,
    ctas_approach=False,
    s3_output=staging_path,
).drop_duplicates()\
 .add_suffix('_fgts')\
 .rename(columns={'cd_perfil_ep_ep':'cd_perfil_fgts',
              'ds_perfil_ep_ep':'ds_perfil_fgts'})
wr.s3.delete_objects(staging_path, boto3_session=boto3_session)

In [ ]:
raw_data.set_index('dt_criacao_chat').groupby(pd.Grouper(freq='W')).count()

In [ ]:
dash_df = []
for tag, values in best_params_per_tag.items():
    print(tag, values)
    try:
        cliente = pd.read_parquet(f'data/{tag}_{list(values.keys())[0]}_merged.parquet')
        display(cliente.head(3))
        agente = pd.read_parquet(f'data/{tag}_{list(values.keys())[1]}_merged.parquet')
        display(cliente.head(3))
    except:
        print('could not read some of the files')
        continue
        
    
    try:
        model_cols = ['ds_entidade',
                      'group',
                      'ds_tags',
                      'ds_message',
                      'dt_criacao_chat',
                      'dt_envio_mensagem',
                      'Topic',
                      'Count',
                      'Name',
                      'Representation',
                      'KeyBERT',
                      'MMR',
                      'Representative_Docs']

        cliente[model_cols] = cliente[model_cols].fillna(agente[model_cols]) 
    except Exception as e:
        print(e)
        cliente['Probability'] = 1
        agente['Probability'] = 1
        model_cols = ['ds_entidade',
                      'group',
                      'ds_tags',
                      'ds_message',
                      'dt_criacao_chat',
                      'dt_envio_mensagem',
                      'Topic',
                      'Count',
                      'Name',
                      'Representation',
                      'KeyBERT',
                      'MMR',
                      'Representative_Docs']

        cliente[model_cols] = cliente[model_cols].fillna(agente[model_cols]) 
        

    qna = cliente.copy()
    qna['Document_before'] = qna.groupby(['id_customer','id_issue']).ds_message.shift(1).astype(str)
    qna['Topic_before'] = qna.groupby(['id_customer','id_issue']).Topic.shift(1).astype(str)

    qna['Document_after'] = qna.groupby(['id_customer','id_issue']).ds_message.shift(-1).astype(str)
    qna['Topic_after'] = qna.groupby(['id_customer','id_issue']).Topic.shift(-1).astype(str)

    qna['chatmonth'] = qna.dt_criacao_chat.dt.to_period('M').dt.to_timestamp()

    qna = pd.merge(qna,
             df_ep,
             how='left',
             left_on=['id_customer','chatmonth'],
             right_on=['customer_id_ep','mes_elegivel_modelo_ep'])

    qna = pd.merge(qna,
             df_fgts,
             how='left',
             left_on=['id_customer','chatmonth'],
             right_on=['customer_id_fgts','mes_elegivel_modelo_fgts'])


    qna_dash = qna[qna.Name.notna()].copy()

    qna_dash['fmt_name'] = qna_dash.Name.apply(lambda x: '2_'+'_'.join(x.replace('-1_','').split('_')[1:])
                                                        if '-1' in x
                                                        else '_'.join(x.replace('-1_','').split('_')[1:])
                              )
    qna_dash['fmt_representation']=qna_dash.KeyBERT.apply(lambda x: ' '.join(x))
    qna_dash['fmt_representative_docs']=qna_dash.Representative_Docs.apply(lambda x: ' '.join(x))
    qna_dash['dt_envio_mensagem']=pd.to_datetime(qna_dash.dt_envio_mensagem, format='%Y-%m-%d')
    qna_dash['fmt_dt_envio_mensagem']=pd.to_datetime(qna_dash.dt_envio_mensagem, format='%Y-%m-%d').dt.floor('D')
    qna_dash['tag'] = tag
    
    print(qna_dash.shape)    
    dash_df.append(qna_dash)


In [ ]:
dashboard = pd.concat(dash_df,axis=0)

In [ ]:
# Define staging_path
staging_path = 

# Define boto3 session
boto3_session = boto3_session = boto3.Session(region_name="sa-east-1")

# Clean temporary folder
wr.s3.delete_objects(staging_path, boto3_session=boto3_session)

query =
# Run query
tmp = wr.athena.read_sql_query(
    sql=query,
    database=None,
    ctas_approach=False,
    s3_output=staging_path,
)
display(tmp)
wr.s3.delete_objects(staging_path, boto3_session=boto3_session)